In [1]:
import hashlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as m
import statistics as s
from random import choices

In [2]:
# Loading in data and finding my row of observations, setting a seed to keep results consistent. 

In [3]:
np.random.seed(7)
mydata = pd.DataFrame(pd.read_csv("exam2.csv", index_col = 0)).loc[hashlib.md5("hjr160230".encode('utf-8')).hexdigest()]
x = mydata.copy().values

First I will start by estimating the upper and lower bounds of the data. Using bootstrapping, we can generate a variance of the lower and upper bounds which we can use to estimate the true parameters. In this case, I am generating 10000 different bootstrapped samples and saving the sampled minimum values in "minvec" and the sampled maximum values in "maxvec".

In [4]:
minvec = []
maxvec = []

for i in range(10000):
    sample = np.random.choice(x,200,replace=True)
    minvec.append(min(sample))
    maxvec.append(max(sample))

Using the values previously generated, we can get a variance corresponding to both the upper and lower bound, and subsequently we can get a standard error as well. 

In [5]:
minvec = pd.DataFrame(minvec).values
maxvec = pd.DataFrame(maxvec).values
minvec.var(),maxvec.var()
stderrmin = minvec.std()
stderrmax = maxvec.std()
stderrmin,stderrmax

(0.9137276112732381, 0.027392348717382498)

We know that the true parameter lies somewhere outside the minimum and maximum values in our data, and we also know that because of the central limit theorem, our minimum and maximum samples follow a normal distribution. To estimate our true lower bound using these two assumptions, we can fit a normal distribution with our lower bound falling on the far right of the curve (at an extreme position, so that the probability of finding the datasets minimum value as the true parameter is less than .0002). Under this distribution, we can say the true mean lies 3.5 standard errors to the left of the lower bound and 3.5 standard errors to the right of the upper bound, calculated below. The final value observed for the true lower bound is -7.233 and the true upper bound is 10.813.

In [6]:
lb = -(3.5*stderrmin)+min(x)
ub = (3.5*stderrmax)+max(x)
lb,ub

(-7.233651507456333, 10.81250419051084)

Now that the lower bound and the upper bound are estimated, we can convert our x values to the scaled z values, so they are bounded between 0 and 1. This will normalize our kumaraswamy distribution, and allow us to best estimate our a and b parameters. 

In [7]:
z = (x - lb)/(ub-lb)

In [8]:
# Create order statistics for both the x data (unscaled) and the z data (scaled)
z_order = np.array(sorted(z.copy()))
x_order = np.array(sorted(x.copy()))

Here I am creating a uniform distribution with bounds 0 and 1, of size 200. Using the uniform distribution, I can 
convert to a kumaraswamy distribution and use this as a test dataset for parameter estimates where they are known
ahead of time. 

In [9]:
unif = np.random.uniform(low=0.0,high=1.0,size = 200)
# setting known parameters to estimate. 
truea = 2
trueb = 3
# converting our uniform distribution to kumaraswamy. 
kumar = (1 - (1 - unif)**(1/trueb))**(1/truea)
kumar = sorted(kumar)
kumar = np.array(kumar)

Here I am setting up vectors to contain different values for a and b, stepping by .1, and looping to generate errors
when comparing my true data to my test data. It should be noted that the test data and the true data being compared are sorted, so the smallest values are compared with similar smaller values, and the errors are squared to account for negative and postive values offsetting each other. The uniform distribution for the tests data changes every time in an 
effort to simulate the fact that the underlying values are unknown and the "parent" uniform values are different.  

In [10]:
avec = np.arange(1.5, 2.5, 0.1)
bvec = np.arange(2.5, 3.5, 0.1)
outp = []
for i in range(5000):
    unif2 = np.random.uniform(low=0.0,high=1.0,size = 200)
    errvec = 20
    outp2 = []
    for a in range(len(avec)):
        for b in range(len(bvec)):
            testkumar = (1 - (1 - unif2)**(1/bvec[b]))**(1/avec[a])
            testkumar = sorted(testkumar)
            testkumar = np.array(testkumar)
            err = ((kumar - testkumar)**2).sum()
            if err < errvec:
                errvec = err
                # for each uniform distribution, we only save the errors if the a and b combination has a smaller error than
                # the previous combination. This way, the last estimates in the outp2 variable has the smallest error. 
                outp2 += [[avec[a],bvec[b],errvec]]
    outp += [outp2[-1]]
outp = pd.DataFrame(outp)

In [11]:
outp.columns = ['a','b','err']
outp.sort_values(by='err')
(outp.loc[outp['err'] <= .025]).mean()

a      2.002439
b      3.131707
err    0.022583
dtype: float64

After generating errors against 5000 different uniform distributions, we can average the a and b values at which we 
find errors under a certain threshold. Because we know the true parameters ahead of time, we have quite small errors
and can choose a small threshold. I suspect using this method against the true dataset will have a larger error and the thrreshold will have to be larger. Here we estimate a to be ~1.96 and be to be ~3.05. These values are pretty close to the true parameters defined above at 2 and 3, so this method seems to be effective for estimating the parameters of a kumaraswamy distribution. NOTE: despite setting a seed in the program, the random uniform distributions seem to be different every time the loop is run, and the estimates stated above may not be the same as what is output when run a different time. In all of the tests, however, the values still estimate the true parameters closely. 

Now we can implement this strategy on the real dataset provided. We are essentially doing the same thing here: creating a random unifrom distribution, converting it to a kumaraswamy distribution using a variety of test values for a and b, and generating an error term between the true data (z_order) and the test data (testkumar). In outp, I am saving the combination of a and b that generates the smallest error. This outp list holds 100 different smallest errors for each random uniform distribution. 

In [12]:
avec = np.arange(1, 10, 0.5)
bvec = np.arange(1, 10, 0.5)
outp = []
outp2 = []
for i in range(100):
    unif2 = np.random.uniform(low=0.0,high=1.0,size = 200)
    errvec = 20
    for a in range(len(avec)):
        for b in range(len(bvec)):
            testkumar = (1 - (1 - unif2)**(1/bvec[b]))**(1/avec[a])
            testkumar = sorted(testkumar)
            testkumar = np.array(testkumar)
            err = ((z_order - testkumar)**2).sum()
            if err < errvec:
                errvec = err
                outp2 += [[avec[a],bvec[b],errvec]]
    outp += [outp2[-1]]
outp = pd.DataFrame(outp)

After a bit of formatting, we can take an average of a and b values where the error falls under a certain threshold, here I chose 1.1 (still quite a small squared error) arbitrarily. We see that our estimates put a at ~9 and b at ~2

In [14]:
outp.columns = ['a','b','err']
outp.sort_values(by='err')
(outp.loc[outp['err'] <= 1.1]).mean()

a      9.231481
b      2.064815
err    0.994825
dtype: float64

Knowing that our a is somewhere around 9 and our b is somewhere around 2, I have tightened the bounds and lowered the step to find a more accurate estimation. Additionally, I have increased the number of random distributions tested against to find more estimated errors. 

In [15]:
avec = np.arange(7, 10, 0.1)
bvec = np.arange(1, 3, 0.1)
outp = []
for i in range(2000):
    unif2 = np.random.uniform(low=0.0,high=1.0,size = 200)
    errvec = 20
    outp2 = []
    for a in range(len(avec)):
        for b in range(len(bvec)):
            testkumar = (1 - (1 - unif2)**(1/bvec[b]))**(1/avec[a])
            testkumar = sorted(testkumar)
            testkumar = np.array(testkumar)
            err = ((z_order - testkumar)**2).sum()
            if err < errvec:
                errvec = err
                outp2 += [[avec[a],bvec[b],errvec]]
    outp += [outp2[-1]]
outp = pd.DataFrame(outp)

With smaller steps and more estimates (2000 compared to 100) we can get a more accurate estimate for a and b. Here we see a is estimated at ~9.6 and b is estimated at ~2.2. These are my final estimations for the a and b parameters for the kumaraswamy distribution. NOTE: here again, the distributions are different every time the loop is ran, so different estimates may be seen when running code again. I am still  able to get similar estimates in every trial, so this method should still properly approximate the true parameters. 

In [16]:
outp.columns = ['a','b','err']
outp.sort_values(by='err')
(outp.loc[outp['err'] <= 1.1]).mean()
estimates = outp.loc[outp['err'] <= 1.1].mean()
a = estimates[0]
b = estimates[1]
a,b

(9.550638977635655, 2.191054313099043)

# Cross Validation
Sumit Paul and I decided to share our estimation methods with each other to cross validate our estimates. Below is the estimation method he used on my data set. This process was only for validation, and the method shown above should be taken as my true estimations and thought process on the problem. 

For this estimation method, Sumit used the median and mean formulas from the Kumaraswamy wikipedia page to estimate a and b. We are comparing the median of the true dataset to a generated median using a test Kumaraswamy dataset, and testing for different values of a and b. 

In [17]:
zmed = np.median(z)
zmean = z.mean()

Here we are validating the similarity between using the np.median (i.e. calculating the median the traditional way, as the center of the data) and the median formula for Kumaraswamy distributions. At each value of a and b, we are getting an average error of .002, meaning that the median values differ only slightly. This means we can use the "traditional" median of our dataset to estimate a and b by comparing it with the Kumaraswamy median and minimizing error. I also have created upper and lower bounds on the error of the median to account for the differences in calculation between Kumaraswamy median and traditional median. When estimating error  on the true dataset, I will look for error values between those upper and lower bounds. 

In [18]:
n = 200
medianerr = []

for a in range(1,100):
    for b in range(1,100):
        x1 = np.random.uniform(size=(n,))
        kumar = (1-((1-x1)**(1/b)))**(1/a)
        kumarmed = (1-(2**(-1/b)))**(1/a)
        medianerr.append(abs(np.median(kumar) - kumarmed))
avgerrmed = np.mean(medianerr)
avgerrmed

0.0021790696636603147

In [19]:
stdevmed = s.stdev(medianerr)
uppermed = avgerrmed + 1.645*stdevmed
lowermed = avgerrmed - 1.645*stdevmed

Here again, we are validating the similarity between the traditional calculation of the mean against the Kumaraswamy calculation of the mean. Similarly to the median, we see an average error of .001, indicating that the means differ only slightly. Upper and lower bounds are generated here as well. 

In [20]:
meanerr = []

for a in range(1,100):
    for b in range(1,100):
        x1 = np.random.uniform(size=(n,))
        kumar = (1-((1-x1)**(1/b)))**(1/a)
        kumarmean = (b*(m.gamma(1+(1/a)))*m.gamma(b))/(m.gamma(1+(1/a)+b))
        meanerr.append(abs(np.mean(kumar) - kumarmean))
avgerrmean = np.mean(meanerr)
avgerrmean

0.0018250516339257785

In [21]:
stdevmean = s.stdev(meanerr)
uppermean = avgerrmean + 1.645*stdevmean
lowermean = avgerrmean - 1.645*stdevmean

In [22]:
avec = np.arange(1, 10, 0.1)
bvec = np.arange(1, 10, 0.1)
errtest = 20
outp2 = []

for a in range(len(avec)):
    for b in range(len(bvec)):
        testmed = (1-(2**(-1/bvec[b])))**(1/avec[a])
        err = abs(zmed - testmed)
        if err > lowermed and err < uppermed:
            errtest = err
            outp2 += [[avec[a],bvec[b],err]]
outp2 = pd.DataFrame(outp2)

In [23]:
outp2.columns = ['a','b','err']
outp2.mean(0)

a      7.846053
b      2.087500
err    0.003414
dtype: float64

In [24]:
avec = np.arange(1, 10, 0.1)
bvec = np.arange(1, 10, 0.1)
errtest = 20
outp3 = []

for a in range(len(avec)):
    for b in range(len(bvec)):
        testmean = (bvec[b]*(m.gamma(1+(1/avec[a])))*m.gamma(bvec[b]))/(m.gamma(1+(1/avec[a])+bvec[b]))
        err = abs(zmean - testmean)
        if err > lowermean and err < uppermean:
            errtest = err
            outp3 += [[avec[a],bvec[b],err]]
outp3 = pd.DataFrame(outp3)

In [25]:
outp3.columns = ['a','b','err']
outp3.mean(0)

a      8.210204
b      1.744898
err    0.002711
dtype: float64

To summarize this process, I wound up getting an estimated a value of 7.85 and b value of 2.08 for the median estimation and an a value of 8.2 and b value of 1.74 for the mean estimation. Comparing these values to my original approach, we can further validate that the true b parameter is in fact around 2. In contrast, the a value has a lot of variation between the 3 methods. Again, this method was only for cross validation and my original approach should be treated as my best estimate for this assignment. 